<a href="https://colab.research.google.com/github/germancruzram/GoPRO_metadato_extractor/blob/main/GoPRO_metadato_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HERRAMIENTA PARA EXTRAER METADATOS DE VIDEOS MP4 DE CAMARAS GOPRO.**



Ing. German Ahmed Cruz Ramírez. 🇳  
ahmedcr_net@hotmail.com

**INFORMACION QUE SE EXTRAE DEL MP4 - GoPRO:** 📸


1.   **IMG TIMESTAMPS** Tiempos en segundos de cada fotograma del video, usados para sincronizar datos.

2.   **SHUT (Shutter):** Información sobre el obturador, como la velocidad de obturación.

3.   **ACCL (Accelerometer):** Medidas de aceleración en tres ejes (X, Y, Z), útiles para analizar movimiento.

4.   **GYRO (Gyroscope):** Velocidad angular en tres ejes, importante para entender la rotación de la cámara.

5.   **GPS5:** Datos de posición GPS, incluyendo latitud, longitud, y altitud.

6.   **GRAV (Gravity Vector):** Vector de gravedad, usado para determinar la orientación respecto a la gravedad.

7.   **MAGN (Magnetometer):** Medidas del campo magnético en tres ejes, para determinar la dirección cardinal.

8.   **CORI (Corrected Orientation):** Orientación corregida de la cámara.

9.   **IORI (Initial Orientation):** Orientación inicial de la cámara al inicio de la grabación.

10.   **GPSP (GPS Precision):** Precisión de las lecturas de GPS.

11.   **GPSF (GPS Fix):** Estado del "fix" de GPS, indicando si la posición es válida.

12.   **GPSU (GPS Update):** Frecuencia de actualización del GPS

# **1. Instalando las librerias necesarias**

In [ ]:
!pip install telemetry-parser
!pip install py-gpmf-parser # install the missing module
from py_gpmf_parser.gopro_telemetry_extractor import GoProTelemetryExtractor
import os
import json
import glob
import py_gpmf_parser as pgfp
import numpy as np

# **2. Testeando el video MP4**

In [ ]:
import subprocess

def extract_gps_from_mp4(mp4_file):
  # Ejecuta exiftool para extraer metadatos GPS
  result = subprocess.run(['exiftool', '-gpslatitude', '-gpslongitude', mp4_file], stdout=subprocess.PIPE, text=True)
  output = result.stdout

  # Procesa la salida para obtener las coordenadas
  gps_data = {}
  for line in output.splitlines():
      if 'GPS Latitude' in line:
          gps_data['latitude'] = line.split(': ')[1].strip()
      elif 'GPS Longitude' in line:
          gps_data['longitude'] = line.split(': ')[1].strip()

  return gps_data

# Especifica la ruta a tu archivo MP4
mp4_file = '/content/GX016233.MP4'

# Extrae datos GPS
gps_data = extract_gps_from_mp4(mp4_file)
print(gps_data)

{'latitude': '18 deg 32\' 14.64" S', 'longitude': '67 deg 54\' 16.56" W'}


# **3. Creando un archivo formato JSON con la información extraida**

In [ ]:
# Ruta de tu archivo de video
video_path = "/content/GX016233.MP4"

# Creamos el extractor
extractor = GoProTelemetryExtractor(video_path)

# Abrimos el archivo fuente
extractor.open_source()

# Extraemos varios tipos de datos
data_types = ["SHUT", "ACCL", "GYRO", "GPS5", "GRAV", "MAGN", "CORI", "IORI", "GPSP", "GPSF", "GPSU"]
extracted_data = {}

for data_type in data_types:
    data, timestamps = extractor.extract_data(data_type)
    if data is not None:
        extracted_data[data_type] = {
            "data": data.tolist() if hasattr(data, 'tolist') else data,
            "timestamps": timestamps.tolist() if hasattr(timestamps, 'tolist') else timestamps
        }

# Obtenemos las marcas de tiempo de las imágenes
image_timestamps = extractor.get_image_timestamps_s()
extracted_data["IMAGE_TIMESTAMPS"] = image_timestamps.tolist() if hasattr(image_timestamps, 'tolist') else image_timestamps

# Cerramos el extractor
extractor.close_source()

# Guardamos los datos extraídos en un archivo JSON
output_json_path = "/content/GX016233_telemetry.json"
with open(output_json_path, 'w') as f:
    json.dump(extracted_data, f)

print(f"Datos de telemetría extraídos y guardados en {output_json_path}")

# Mostramos un resumen de los datos extraídos
for data_type, data in extracted_data.items():
    if isinstance(data, dict) and "data" in data:
        print(f"{data_type}: {len(data['data'])} puntos de datos")
    elif isinstance(data, list):
        print(f"{data_type}: {len(data)} puntos de datos")

Datos de telemetría extraídos y guardados en /content/GX016233_telemetry.json
SHUT: 2159 puntos de datos
ACCL: 14491 puntos de datos
GYRO: 14491 puntos de datos
GPS5: 720 puntos de datos
GRAV: 2159 puntos de datos
MAGN: 0 puntos de datos
CORI: 2159 puntos de datos
IORI: 2159 puntos de datos
GPSP: 72 puntos de datos
GPSF: 72 puntos de datos
GPSU: 0 puntos de datos
IMAGE_TIMESTAMPS: 2159 puntos de datos


In [ ]:
class GoProTelemetryExtractor:
    def __init__(self, mp4_filepath):
        self.mp4_filepath = mp4_filepath
        self.handle = None

    def open_source(self):
        if self.handle is None:
            self.handle = pgfp.OpenMP4Source(self.mp4_filepath, pgfp.MOV_GPMF_TRAK_TYPE, pgfp.MOV_GPMF_TRAK_SUBTYPE, 0)
        else:
            raise ValueError("Source is already opened!")

    def close_source(self):
        if self.handle is not None:
            pgfp.CloseSource(self.handle)
            self.handle = None
        else:
            raise ValueError("No source to close!")

    def get_image_timestamps_s(self):
        if self.handle is None:
            raise ValueError("Source is not opened!")

        num_frames, numer, denom = pgfp.GetVideoFrameRateAndCount(self.handle)
        frametime = denom / numer
        timestamps = []
        for i in range(num_frames):
            timestamps.append(i * frametime)
        return np.array(timestamps)

    def extract_data(self, sensor_type):
        if self.handle is None:
            raise ValueError("Source is not opened!")

        results = []
        timestamps = []

        rate, start, end = pgfp.GetGPMFSampleRate(self.handle, pgfp.Str2FourCC(sensor_type), pgfp.Str2FourCC("SHUT"))

        num_payloads = pgfp.GetNumberPayloads(self.handle)
        for i in range(num_payloads):
            payloadsize = pgfp.GetPayloadSize(self.handle, i)
            res_handle = 0
            res_handle = pgfp.GetPayloadResource(self.handle, res_handle, payloadsize)
            payload = pgfp.GetPayload(self.handle, res_handle, i, payloadsize)

            ret, t_in, t_out = pgfp.GetPayloadTime(self.handle, i)

            delta_t = t_out - t_in

            ret, stream = pgfp.GPMF_Init(payload, payloadsize)
            if ret != pgfp.GPMF_ERROR.GPMF_OK:
                continue

            while pgfp.GPMF_ERROR.GPMF_OK == pgfp.GPMF_FindNext(stream, pgfp.Str2FourCC("STRM"), pgfp.GPMF_RECURSE_LEVELS_AND_TOLERANT):
                if pgfp.GPMF_ERROR.GPMF_OK != pgfp.GPMF_FindNext(stream, pgfp.Str2FourCC(sensor_type), pgfp.GPMF_RECURSE_LEVELS_AND_TOLERANT):
                    continue

                key = pgfp.GPMF_Key(stream)
                elements = pgfp.GPMF_ElementsInStruct(stream)
                rawdata = pgfp.GPMF_RawData(stream)
                samples = pgfp.GPMF_Repeat(stream)
                if samples:
                    buffersize = samples * elements * 8
                    ret, data = pgfp.GPMF_ScaledData(stream, buffersize, 0, samples, pgfp.GPMF_SampleType.DOUBLE)
                    data = data[:samples * elements]
                    if pgfp.GPMF_ERROR.GPMF_OK == ret:
                        results.extend(np.reshape(data, (-1, elements)))
                        timestamps.extend([t_in + i * delta_t / samples for i in range(samples)])
            pgfp.GPMF_ResetState(stream)

        return np.array(results), np.array(timestamps) + start

    def extract_data_to_json(self, json_file, sensor_types=["ACCL", "GYRO", "GPS5"]):
        out_dict = {"img_timestamps_s": self.get_image_timestamps_s().tolist()}
        for sensor in sensor_types:
            data, timestamps = self.extract_data(sensor)
            out_dict.update({sensor: {"data": data.tolist(), "timestamps_s": timestamps.tolist()}})
        with open(json_file, "w") as f:
            json.dump(out_dict, f)

# Ruta de tu archivo de video
video_path = "/content/GX016233.MP4"

# Creamos el extractor
extractor = GoProTelemetryExtractor(video_path)

# Abrimos el archivo fuente
extractor.open_source()

# Guardamos los datos extraídos en un archivo JSON
output_json_path = "/content/GX016233_telemetry.json"
extractor.extract_data_to_json(output_json_path, sensor_types=["SHUT", "ACCL", "GYRO", "GPS5", "GRAV", "MAGN", "CORI", "IORI", "GPSP", "GPSF", "GPSU"])

# Cerramos el extractor
extractor.close_source()

print(f"Datos de telemetría extraídos y guardados en {output_json_path}")

# Mostramos un resumen de los datos extraídos
with open(output_json_path, 'r') as f:
    extracted_data = json.load(f)
    for data_type, data in extracted_data.items():
        if isinstance(data, dict) and "data" in data:
            print(f"{data_type}: {len(data['data'])} puntos de datos")
        elif isinstance(data, list):
            print(f"{data_type}: {len(data)} puntos de datos")

Datos de telemetría extraídos y guardados en /content/GX016233_telemetry.json
img_timestamps_s: 2159 puntos de datos
SHUT: 2159 puntos de datos
ACCL: 14491 puntos de datos
GYRO: 14491 puntos de datos
GPS5: 720 puntos de datos
GRAV: 2159 puntos de datos
MAGN: 0 puntos de datos
CORI: 2159 puntos de datos
IORI: 2159 puntos de datos
GPSP: 72 puntos de datos
GPSF: 72 puntos de datos
GPSU: 0 puntos de datos


# **4. Construyendo formatos estandar GPX y NMEA con las posiciones GPS**

In [ ]:
import json
import gpxpy
import gpxpy.gpx
from datetime import datetime, timezone

# Cargar el archivo JSON de telemetría
try:
  with open('/content/GX016233_telemetry.json', 'r') as f:
      telemetry_data = json.load(f)
except FileNotFoundError:
  print("El archivo de telemetría no se encontró. Asegúrate de que el archivo existe en la ubicación correcta.")
  exit()

# Imprimir un resumen de las claves principales del JSON
print("Claves principales en el JSON:", list(telemetry_data.keys()))

# Verificar si las claves existen
gps_data = telemetry_data.get('GPS5', {}).get('data')
gps_timestamps = telemetry_data.get('GPS5', {}).get('timestamps_s')  # Cambiado a 'timestamps_s'

if gps_data is None or gps_timestamps is None:
  print("Los datos GPS o los timestamps no se encontraron en el archivo JSON.")
  exit()

# 1. Crear archivo GPX
gpx = gpxpy.gpx.GPX()
gpx_track = gpxpy.gpx.GPXTrack()
gpx.tracks.append(gpx_track)
gpx_segment = gpxpy.gpx.GPXTrackSegment()
gpx_track.segments.append(gpx_segment)

for timestamp, gps in zip(gps_timestamps, gps_data):
  # Convertir el timestamp a datetime
  dt = datetime.fromtimestamp(timestamp, timezone.utc)

  # Crear un punto GPX
  gpx_point = gpxpy.gpx.GPXTrackPoint(
      latitude=gps[0],
      longitude=gps[1],
      elevation=gps[2],
      time=dt
  )

  # Añadir el punto al segmento
  gpx_segment.points.append(gpx_point)

# Guardar el archivo GPX
with open('/content/gps_route.gpx', 'w') as gpxfile:
  gpxfile.write(gpx.to_xml())

print("Archivo GPX creado: /content/gps_route.gpx")

# 2. Crear archivo NMEA
def create_nmea_sentence(lat, lon, alt, timestamp):
  dt = datetime.fromtimestamp(timestamp, timezone.utc)
  lat_deg = int(abs(lat))
  lat_min = (abs(lat) - lat_deg) * 60
  lon_deg = int(abs(lon))
  lon_min = (abs(lon) - lon_deg) * 60
  lat_hemisphere = 'N' if lat >= 0 else 'S'
  lon_hemisphere = 'E' if lon >= 0 else 'W'

  nmea = f"$GPGGA,{dt.strftime('%H%M%S')},{lat_deg:02d}{lat_min:07.4f},{lat_hemisphere},{lon_deg:03d}{lon_min:07.4f},{lon_hemisphere},1,08,1.0,{alt:.1f},M,0.0,M,,"
  checksum = 0
  for char in nmea[1:]:
      checksum ^= ord(char)
  return f"{nmea}*{checksum:02X}\n"

with open('/content/gps_route.nmea', 'w') as nmeafile:
  for timestamp, gps in zip(gps_timestamps, gps_data):
      nmeafile.write(create_nmea_sentence(gps[0], gps[1], gps[2], timestamp))

print("Archivo NMEA creado: /content/gps_route.nmea")

Claves principales en el JSON: ['img_timestamps_s', 'SHUT', 'ACCL', 'GYRO', 'GPS5', 'GRAV', 'MAGN', 'CORI', 'IORI', 'GPSP', 'GPSF', 'GPSU']
Archivo GPX creado: /content/gps_route.gpx
Archivo NMEA creado: /content/gps_route.nmea


# **5. CREANDO UN MAPA CON LA UBICACION CON LOS METADATOS DEL VIDEO**

In [ ]:
import folium
import gpxpy

# Ruta al archivo GPX
gpx_file_path = '/content/gps_route.gpx'

# Leer el archivo GPX
with open(gpx_file_path, 'r') as gpx_file:
  gpx = gpxpy.parse(gpx_file)

# Verificar si hay datos en el GPX
if gpx.tracks and gpx.tracks[0].segments:
  first_track = gpx.tracks[0]
  first_segment = first_track.segments[0]
  start_point = first_segment.points[0]
  map_center = [start_point.latitude, start_point.longitude]
else:
  map_center = [0, 0]  # Fallback si no hay datos

# Crear el mapa centrado en el primer punto
mymap = folium.Map(location=map_center, zoom_start=13)

# Añadir la ruta GPX al mapa
for track in gpx.tracks:
  for segment in track.segments:
      points = [(point.latitude, point.longitude) for point in segment.points]
      folium.PolyLine(points, color='blue', weight=2.5, opacity=1).add_to(mymap)

# Mostrar el mapa en el notebook
mymap
